In [2]:
import pandas as pd
import numpy as np
from pathlib import Path

import config

import importlib
try:
    importlib.reload(config) # reload module
except NameError:
    pass

# DF Vote

In [11]:
df_vote = pd.read_excel(config.RAW_DATA_PATH, sheet_name="vote")

vote_count_regex = "\d+\.\d+$|\d+\,\d+$|\d+$"

map_partai = {
    "Partai Kebangkitan Bangsa": "PKB",
    "Partai Gerakan Indonesia Raya": "Gerindra",
    "Partai Demokrasi Indonesia Perjuangan": "PDIP",
    "Partai Golongan Karya": "Golkar",
    "Partai Nasdem": "Nasdem",
    "Partai Gerakan Perubahan Indonesia": "Garuda",
    "Partai Berkarya": "Berkarya",
    "Partai Keadilan Sejahtera": "PKS",
    "Partai Persatuan Indonesia": "Perindo",
    "Partai Persatuan Pembangunan": "PPP",
    "Partai Solidaritas Indonesia": "PSI",
    "Partai Amanat Nasional": "PAN",
    "Partai Hati Nurani Rakyat": "Hanura",
    "Partai Demokrat": "Demokrat",
    "Partai Bulan Bintang": "PBB",
    "Partai Keadilan dan Persatuan Indonesia": "PKP",
}

partai_no_cat = pd.CategoricalDtype(list(map_partai.values()), ordered=True)


def clean_df_vote(df):
    df_tem = (
        df.loc[~(df["values"].isna())]
        .assign(
            tipe=lambda df_: np.where(
                df_["values"].str.startswith("A.1"), "Partai", "Calon"
            ),
            values=lambda df_: (
                df_["values"]
                .str.replace("^A.1. |^A.1 ", "", regex=True)  # remove A1
                .str.replace("^A.2. |^A.2 ", "", regex=True)  # remove A1
                .str.replace("^\d+\.", "", regex=True)  # remove no urut
                .str.strip()
            ),
            nama=lambda df_: (
                df_["values"]
                .str.replace(vote_count_regex, "", regex=True)  # remove vote
                .str.replace("\d+\.\d+|\d+", "", regex=True)  # remove digit
                .str.strip()
            ),
            vote=lambda df_: (
                df_["values"]
                .str.extract(f"({vote_count_regex})")[0]
                .str.replace(".", "", regex=False)
                .str.replace(",", "", regex=False)
                .str.strip()
                .astype(int)
            ),
            partai=lambda df_: np.where(df_["tipe"] == "Partai", df_["nama"], np.NaN),
            terpilih=lambda df_: df_["terpilih"].fillna(0),
        )
        .assign(
            partai=lambda df_: df_["partai"]
            .fillna(method="ffill")
            .map(map_partai)
            .astype(partai_no_cat),
            no_urut=lambda df_: df_.groupby(["dapil_no", "partai"]).transform(
                "cumcount"
            ),
        )
        .drop(columns=["values"])
    )

    partai_vote = df_tem.loc[
        df_tem["tipe"] == "Partai", ["dapil_no", "dapil_nama", "nama", "vote"]
    ].rename(columns={"vote": "partai_vote"})

    df_res = (
        df_tem.merge(
            right=partai_vote, how="left", on=["dapil_no", "dapil_nama", "nama"]
        )
        .assign(
            partai_vote=lambda df_: df_["partai_vote"].fillna(method="ffill"),
            dapil_nama=lambda df_: df_["dapil_no"].astype(int).map({
                1: "Jakarta Pusat",
                2: "Jakarta Utara A & Kab. Kep. Seribu",
                3: "Jakarta Utara B",
                4: "Jakarta Timur A",
                5: "Jakarta Timur B",
                6: "Jakarta Timur C",
                7: "Jakarta Selatan A",
                8: "Jakarta Selatan B",
                9: "Jakarta Barat A",
                10: "Jakarta Barat B",
            })
            )
        .loc[lambda df_: df_["tipe"] == "Calon"]
        .loc[
            :,
            [
                "dapil_no",
                "dapil_nama",
                "no_urut",
                "nama",
                "vote",
                "partai",
                "partai_vote",
                "terpilih",
            ],
        ]
    )
    return df_res


df_vote_clean = clean_df_vote(df_vote)

# # check vote for each partai to make sure
# (df_vote_clean
#     .loc[df_vote_clean['tipe'] == 'Partai']
#     .groupby(['dapil_no', 'dapil_nama', 'nama'])
#     .agg(vote=('vote', 'sum'))
#     .reset_index()
#     .pivot(index='nama', columns=['dapil_no', 'dapil_nama'], values='vote')
# )

# check vote for each partai + calon to make sure
# (df_vote_clean
#     .groupby(['dapil_no', 'dapil_nama', 'partai'])
#     .agg(vote=('vote', 'sum'))
#     .reset_index()
#     .loc[lambda df_: df_['vote'] > 0]
#     .loc[lambda df_: df_['dapil_no'] == 10]
# )

# make sure that partai has no urut 0
# (df_vote_clean
#     .loc[df_vote_clean['tipe'] == 'Partai', 'no_urut']
#     .sum()
# )

# check terpilih one by one
# (df_vote_clean
#     .loc[(df_vote_clean['terpilih'] == True) & (df_vote_clean['dapil_no'] == 10), ['partai', 'no_urut', 'nama']]
# )

# DF Result

In [12]:
df_res = pd.read_excel(config.RAW_DATA_PATH, sheet_name='result')

partai_no = [
    'Partai Kebangkitan Bangsa',
    'Partai Gerakan Indonesia Raya',
    'Partai Demokrasi Indonesia Perjuangan',
    'Partai Golongan Karya',
    'Partai Nasdem',
    'Partai Gerakan Perubahan Indonesia',
    'Partai Berkarya',
    'Partai Keadilan Sejahtera',
    'Partai Persatuan Indonesia',
    'Partai Persatuan Pembangunan',
    'Partai Solidaritas Indonesia',
    'Partai Amanat Nasional',
    'Partai Hati Nurani Rakyat',
    'Partai Demokrat',
    'Partai Bulan Bintang',
    'Partai Keadilan dan Persatuan Indonesia'
]
partai_regex = '|'.join(partai_no)

def clean_df_res(df):
    return (df
        .assign(
            nama=lambda df_: (df_['values']
                            .str.replace(partai_regex, '', regex=True) # remove partai
                            .str.replace('^\w+ ? \w+', '', regex=True) # remove beginning digit
                            .str.replace('\d+, ?\d+ \d+$', '', regex=True) # remove after digit
                            .str.replace('\d', '', regex=True)
                            .str.replace(',', '', regex=False)
                            .str.strip()
                            ),
            no_urut=lambda df_: (df_['values']
                                 .str.extract('(^\d+ \d?)') # extract beginning digit
                                 ),
            partai=lambda df_: df_['values'].str.extract(f'({partai_regex})'), 
            vote=lambda df_: (df_['values']
                            .str.replace(partai_regex, '', regex=True) # remove partai
                            .str.strip()
                            .str.replace('^\w+ ? \w+', '', regex=True) # remove beginning digit
                            .str.replace(' \d{1}$', '', regex=True) # remove last sole digit
                            .str.replace('\D', '', regex=True) # remove nondigit
                            .astype(int)
                            )
        )
        .assign(
            partai=lambda df_: df_['partai'].fillna(method='ffill'), 
            no_urut=lambda df_: (df_['no_urut']
                                 .str.strip()
                                 .str.split(' ')
                                 .str[-1]
                                 .astype(int)
                                 ) # get last digit
            )
        .drop(columns=["values"])
    )
    
df_res_clean = clean_df_res(df_res)

# check one by one dapil
# (df_res_clean
#     .loc[df_res_clean['dapil_no'] == 10]
# )
# df_res_clean

# Save DF

In [16]:
# path = Path.cwd().parents[0] / 'output/vote_result.xlsx'
# df_vote_clean.to_excel(path, index=False)

In [14]:
def save_multiple_dfs(list_df, list_sheet_name, filepath):
    """save multiple dfs to one file with multiple sheets

    Args:
        list_df (list): list of dataframe objects
        list_sheet_name (list): list of string for sheet name
        filepath (string): path of file
    """
    import xlsxwriter
    filepath= filepath
    writer = pd.ExcelWriter(filepath, engine= 'xlsxwriter')

    for df in list_df:
        df.to_excel(writer, sheet_name= list_sheet_name.pop(0), index= False)

    writer.close()

dapil_nos = df_vote_clean["dapil_no"].unique()
dfs= []
list_sheet_name = []

for no in dapil_nos:
    data = (df_vote_clean
            .loc[df_vote_clean["dapil_no"] == no]
            .rename(columns=lambda c:c.title()
                    .replace("_", " "))
                    .reset_index(drop=True)
            .rename(columns={
                "Dapil No": "No Dapil",
                "Dapil Nama": "Nama Dapil",
                "No Urut": "No Urut Calon",
                "Nama": "Nama Calon",
                "Vote": "Suara Calon",
                "Partai Vote": "Suara Partai",
            })
            .assign(Terpilih=lambda df_: df_["Terpilih"].map({
                0.0: "Tidak Terpilih", 
                1.0: "Terpilih", 
            }))
        )
    dfs.append(data)
    list_sheet_name.append(f"Dapil {no}")

# save_multiple_dfs(
#     dfs, list_sheet_name, Path.cwd().parents[0] / "output/Data Pemilu 2019 Clean.xlsx"
# )

In [17]:
(df_vote_clean["dapil_no"].astype(str) + " " + df_vote_clean["dapil_nama"]).sort_values().unique()

array(['1 Jakarta Pusat', '10 Jakarta Barat B',
       '2 Jakarta Utara A & Kab. Kep. Seribu', '3 Jakarta Utara B',
       '4 Jakarta Timur A', '5 Jakarta Timur B', '6 Jakarta Timur C',
       '7 Jakarta Selatan A', '8 Jakarta Selatan B', '9 Jakarta Barat A'],
      dtype=object)